작성: 2018.04.10

# 연구 목적
1. Object detection & segmentation 학습을 위한 과정으로,
후두내시경 영상(Stroboscopy) 에서,실제 후두 부위만 추출하는 것을 목적으로 함

2. 추후 후두의 병변(pathologic finding)을 text로 description하는 모델을 제작

---
## Stroboscope Spec.

All exams were taken by PENTAX Medical Stroboscopy Systems (https://www.pentaxmedical.com/pentax/en/99/2/Stroboscopy-System)

### Movie Info
Frames per second: Approx. 30 fps  
Size: 720x480x3 (width, height, channel)

### Stroboscopy Movie

!["Stroboscopy"](./report/01.stroboscopy.gif)

----
### Input Data

#### True positive
![Input_true](./report/01_true_images.png)
#### True Negative
![Input_false](./report/01_false_images.png)


----
### GPU
nVidia 1060 (6GB)


---
## Methodology

1. Strobo scopy 영상을 frame별로 나누어, 중앙에서 400x400 pixel로 crop함 
2. 400x400 이미지를 input시 224x224로 변환<br/>
<b>=> 위의 True positive, true negative 이미지와 같음</b><br/>
3. 각 채널당 0~255 의 color 값을 0~1.0 으로 변환함

### Number of samples
Train: 5921 images (True 4115, False 1806)<br/> 
Test:  229 images (True 122, False 107)


### Model
Resnet50 을 이용<br/>
Top Layer를 제거한 뒤 FCN를 바꿔 2가지 class로 분류(True/False)<br/>
Pretrained weight 없이 학습을 시작함<br/>
Batch size: 8 (16부터 out of memory)<br/>

![Model](./report/model.png)


----
### Training

ImageDataGenerator 를 이용해 rotation, width_shift, height_shift, shear, zoom 등을 적용함<br/>
Optimizer = SGD<br/>


---
### Result

![Result](./report/last1.png)
![Result2](./report/last2.png)

## CNN Activation Map 출력

위의 세팅된 모델에서 activation_49 까지의 결과 값을 받은 뒤,<br/>
prediction된 값을 이용하여 FCN을 거꾸로 올라가서 heat map을 작성한다.

Negative로 분류된 이미지 - 실제로 성대를 제외한 부분이 수치가 높게 표시됨
![CAM1](./report/res0320_1.png)

Positive로 분류된 이미지 - 실제 성대의 feature에 heatmap이 표시됨
![CAM2](./report/res0320_2.png)

Postivie로 분류되었으나, 한쪽 성대에만 feature가 찾아진 이미지
![CAM3](./report/res0320_3.png)